In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch3d as p3d
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pytorch3d.structures import Meshes
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.io import load_obj, load_ply, save_ply
from pytorch3d.loss import chamfer_distance

In [2]:
### config

# 定义点云数据的输入维度和输出维度
input_dim = 3  # 每个点的特征维度
output_dim = 3  # 输出点云的特征维度（可以与输入维度相同）

device = "cuda" if torch.cuda.is_available() else "cpu"
num_epochs = 300
batch_size = 15

sxxxxes = os.listdir("../../data/all_results/")
train_sxxxxes = sxxxxes[:372]
test_sxxxxes = sxxxxes[372:]

In [3]:
# 创建一个简单的全连接神经网络类
class PointCloudFCNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PointCloudFCNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)  # 输入层到隐藏层1
        self.fc2 = nn.Linear(64, 128)  # 隐藏层1到隐藏层2
        self.fc3 = nn.Linear(128, 256)  # 隐藏层2到隐藏层3
        self.fc4 = nn.Linear(256, 128)  # 隐藏层3到隐藏层4
        self.fc5 = nn.Linear(128, output_dim)  # 隐藏层4到输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)  # 输出层不使用激活函数
        return x


# DataSet类
class TrainingSet(Dataset):
    def __init__(self, s_name_list, transform=None, sample_num=5000) -> None:
        self.input = [
            f"../../data/all_results/{sname}/0_sphere.obj" for sname in s_name_list
        ]
        self.target = [
            f"../../data/all_results/{sname}/{sname}.ply" for sname in s_name_list
        ]
        self.sample_num = sample_num
        self.transform = transform

    def __len__(self):
        return len(self.input)

    def __getitem__(self, index):
        #对input采样5000个点
        input_tensor_points, input_tensor_faces, _ = load_obj(self.input[index], load_textures=False)
        input_mesh = Meshes([input_tensor_points], [input_tensor_faces.verts_idx]) #注意这里有个.verts_idx
        input_tensor_points_sampled = sample_points_from_meshes(input_mesh,5000).squeeze(0)
        #对output采样5000个点
        target_tensor_points, target_tensor_faces = load_ply(self.target[index])
        target_mesh = Meshes([target_tensor_points],[target_tensor_faces])
        target_tensor_points_sampled = sample_points_from_meshes(target_mesh).squeeze(0)

        return input_tensor_points_sampled.to(device),target_tensor_points_sampled.to(device)

    def get_target(self, input):
        raise NotImplemented

In [4]:
#################################################################################
# pts,faces = load_ply(f"../../data/all_results/s0004_pulmonary_artery.nii.g_1/s0004_pulmonary_artery.nii.g_1.ply")
# mymesh = Meshes([pts],[faces])
# a = sample_points_from_meshes(mymesh,5)
# a = a.squeeze(0)
# a
# mymesh
#################################################################################

In [5]:
def plot_pointcloud(points, title=""):
    """Sample points uniformly from the surface of the mesh."""
    x, y, z = points.clone().detach().cpu().squeeze().unbind(1)
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter3D(x, z, -y)
    ax.set_xlabel("x")
    ax.set_ylabel("z")
    ax.set_zlabel("y")
    ax.set_title(title)
    ax.view_init(190, 30)
    plt.show()

In [6]:
my_train_dataset = TrainingSet(train_sxxxxes)
data_loader = DataLoader(my_train_dataset, batch_size=batch_size, shuffle=False)


In [7]:
# 创建模型实例
model = PointCloudFCNet(input_dim, output_dim).to(device)
model

PointCloudFCNet(
  (fc1): Linear(in_features=3, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=3, bias=True)
)

In [8]:
# 定义损失函数和优化器
criterion = chamfer_distance
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 使用Adam优化器

In [9]:
next(iter(data_loader))

[tensor([[[ 1.1201e-03, -1.2780e-01, -1.5456e-01],
          [ 3.3912e-01, -1.8870e-01, -2.6119e-01],
          [ 2.2809e-01,  2.8302e-02,  1.3687e-01],
          ...,
          [ 3.8677e-01, -2.7883e-01, -2.1206e-01],
          [-5.6391e-01, -3.5518e-01, -1.0207e-01],
          [ 4.0928e-01,  1.0788e-02,  2.7254e-02]],
 
         [[-1.5282e-01, -2.0984e-01,  1.6452e-01],
          [-2.3907e-01,  3.5701e-01, -8.6453e-02],
          [ 4.6404e-01, -2.3882e-01,  1.0162e-01],
          ...,
          [-2.1422e-01,  3.3570e-01, -4.4510e-02],
          [-1.6109e-01,  3.1048e-01, -2.6347e-01],
          [-1.4922e-01,  3.7911e-01, -4.1280e-01]],
 
         [[ 7.1821e-01, -7.7784e-02,  2.4465e-04],
          [ 5.5877e-01,  3.0769e-02, -7.9908e-02],
          [-1.2117e-02, -2.1917e-01,  2.5476e-01],
          ...,
          [ 4.2557e-01, -3.5235e-02, -5.9317e-02],
          [-5.8487e-02,  2.5119e-01,  1.5489e-01],
          [ 1.1165e-01,  3.8742e-01,  3.9525e-02]],
 
         ...,
 
         [[ 

In [10]:
# 训练模型

for epoch in range(num_epochs):
    for input_point_cloud,target_point_cloud in data_loader:
        optimizer.zero_grad()
        output_point_cloud = model(input_point_cloud)  # 前向传播
        loss,_ = criterion(output_point_cloud, target_point_cloud)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 优化模型参数

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    # if (epoch + 1) % 5 == 0:
    #     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        # plot_pointcloud(output_point_cloud)# 绘制点云

Epoch [1/300], Loss: 0.0411
Epoch [2/300], Loss: 0.0153
Epoch [3/300], Loss: 0.0098
Epoch [4/300], Loss: 0.0088
Epoch [5/300], Loss: 0.0083
Epoch [6/300], Loss: 0.0080
Epoch [7/300], Loss: 0.0078
Epoch [8/300], Loss: 0.0077
Epoch [9/300], Loss: 0.0076
Epoch [10/300], Loss: 0.0074
Epoch [11/300], Loss: 0.0074
Epoch [12/300], Loss: 0.0073
Epoch [13/300], Loss: 0.0072
Epoch [14/300], Loss: 0.0070
Epoch [15/300], Loss: 0.0070
Epoch [16/300], Loss: 0.0070
Epoch [17/300], Loss: 0.0069
Epoch [18/300], Loss: 0.0069
Epoch [19/300], Loss: 0.0067
Epoch [20/300], Loss: 0.0068
Epoch [21/300], Loss: 0.0067
Epoch [22/300], Loss: 0.0066
Epoch [23/300], Loss: 0.0066
Epoch [24/300], Loss: 0.0065
Epoch [25/300], Loss: 0.0065
Epoch [26/300], Loss: 0.0064
Epoch [27/300], Loss: 0.0064
Epoch [28/300], Loss: 0.0064
Epoch [29/300], Loss: 0.0063
Epoch [30/300], Loss: 0.0062
Epoch [31/300], Loss: 0.0063
Epoch [32/300], Loss: 0.0062
Epoch [33/300], Loss: 0.0061
Epoch [34/300], Loss: 0.0062
Epoch [35/300], Loss: 0

In [11]:
# 使用训练好的模型生成新的点云数据
try_input_point_cloud,_,_ = load_obj("../../data/all_results/s1272_pulmonary_artery.nii.g_1/0_sphere.obj", load_textures=False)
new_point_cloud = model(try_input_point_cloud.to(device))
new_point_cloud
# 输出的new_point_cloud包含了经过神经网络处理后的新点云数据

tensor([[ 0.0228,  0.0194, -0.1228],
        [ 0.0322, -0.0287, -0.0990],
        [ 0.0581, -0.0345, -0.1035],
        ...,
        [-0.3258,  0.6229, -0.0555],
        [-0.3130,  0.5785, -0.0096],
        [-0.2497,  0.6825, -0.0189]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [1]:
#看一眼新的输出长啥样
save_ply("./9-20-1920.ply",new_point_cloud)

NameError: name 'save_ply' is not defined